In [1]:
import pandas as pd
import os
import time
from urllib.request import urlopen


urls = lambda num: f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={num}&year1=1981&year2=2020&type=Mean'
csvs = []
provinces = ['Черкаська','Чернігівська','Чернівецька','Крим','Дніпровська','Донецька','Івано-Франківська',
                  'Харківська','Херсонська','Хмельницька','Київська','Київ','Кіровоградська','Луганська','Львівська',
                  'Миколаївська','Одеська','Полтавська','Рівненська','Севастополь','Сумська','Тернопільська','Закарпатська',
                  'Вінницька','Волинь','Запорізька','Житомирська']
repl = dict(zip([i for i in range(1,26)], [24,25,5,6,27,23,26,7,11,13,14,15,16,17,18,19,21,22,8,9,10,1,3,2,4]))
current_time = time.ctime()[4:19].replace(':','.') 
for i in range(1,28):
    vhi_url = urlopen(urls(i))
    out = open(f'vhi_id_{i}{current_time}.csv','wb')
    csvs.append(f'vhi_id_{i}{current_time}.csv')
    out.write(vhi_url.read())
    out.close()


In [2]:
cont=[]
for i in range(1,26):
    df = pd.read_csv(f'vhi_id_{i}{current_time}.csv', header=1, names=['year', 'week', 'NDVI', 'BT', 'VCI', 'TCI', 'VHI','Region', 'NaN'])
    del df['NaN']
    df.drop([2026], inplace=True)
    df['Region'] = f'{provinces[repl[int(i)]-1]} область'
    cont.append(df)
alldata=pd.concat(cont)
alldata1=alldata[alldata['VHI']>-1]
alldata1


,year,week,NDVI,BT,VCI,TCI,VHI,Region
32,1981,35.0,0.255,292.68,13.75,73.72,43.74,Вінницька область
33,1981,36.0,0.255,292.31,19.82,63.72,41.77,Вінницька область
34,1981,37.0,0.258,291.88,30.21,51.66,40.93,Вінницька область
35,1981,38.0,0.259,291.08,39.91,42.64,41.28,Вінницька область
36,1981,39.0,0.256,289.75,47.88,37.60,42.74,Вінницька область
...,...,...,...,...,...,...,...,...
2073,2020,48.0,0.139,272.46,62.49,26.15,44.32,Крим область
2074,2020,49.0,0.125,270.68,60.09,27.23,43.66,Крим область
2075,2020,50.0,0.113,269.47,56.87,24.97,40.92,Крим область
2076,2020,51.0,0.104,268.52,56.18,22.78,39.48,Крим область


In [3]:
pd.options.display.max_rows=100
data = alldata1[['Region',"year","VHI"]].groupby(['Region',"year"])

data.min()

VHI
Region                    year       
Івано-Франківська область 1981  44.05
                          1982  28.77
                          1983  30.10
                          1984  20.89
                          1985  26.66
...                               ...
Чернігівська область      2016  31.59
                          2017  38.22
                          2018  36.64
                          2019  20.26
                          2020  34.48

[1000 rows x 1 columns]

In [61]:
data_extreme_drought=data.min()[data.min()['VHI']<15]
data_extreme_drought

VHI
Region                    year       
Івано-Франківська область 2000   9.36
Вінницька область         2000  10.68
Донецька область          2007  13.28
                          2012  13.54
Закарпатська область      1986  14.99
                          1993   6.26
Київська область          2003  14.53
                          2007  12.23
Крим область              1994  11.91
Луганська область         2000  10.60
Львівська область         1999  12.88
                          2000   6.49
Одеська область           1986  12.45
                          1993  13.10
Рівненська область        2007   5.94
Сумська область           2007   5.52
Херсонська область        2000   8.13
Хмельницька область       1984  12.64
Чернігівська область      2000  11.25

In [62]:
pd.options.display.max_rows=547
data_moderate_drought=data.min()[(data.min()['VHI']>15)&(data.min()['VHI']<35)]
data_moderate_drought

VHI
Region                    year       
Івано-Франківська область 1982  28.77
                          1983  30.10
                          1984  20.89
                          1985  26.66
                          1986  16.38
...                               ...
Чернігівська область      2012  29.28
                          2013  32.92
                          2015  19.94
                          2016  31.59
                          2019  20.26

[645 rows x 1 columns]

In [64]:
def drought_regions():
    year = input("Введите год: ")
    try:
        print(f"Экстримальные засухи в {year} были в таких областях: ")
        for region in data_extreme_drought.xs(year, level=1).index:
            print(region)
    except KeyError:
        print(f"В {year} году не было экстримальных засух.")
    try:
        print(f"\nУмеренные засухи в {year} были в таких областях: ")
        for region in data_moderate_drought.xs(year, level=1).index:
            print(region)
    except KeyError:
        print(f"В {year} году не было умеренных засух.")
drought_regions()   

Введите год: 2020
Экстримальные засухи в 2020 были в таких областях: 
В 2020 году не было экстримальных засух.

Умеренные засухи в 2020 были в таких областях: 
Вінницька область
Донецька область
Тернопільська область


In [9]:

for j in csvs: os.remove(j)
    
